In [25]:
# Импорт необходимых библиотек
import pandas as pd
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Trainer, TrainingArguments
import torch
import os
from sklearn.model_selection import train_test_split

# Путь к папке с аудиофайлами
audio_dir = '/kaggle/input/bengaliai-speech/examples/'

# Получение списка аудиофайлов
audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.wav')]

# Создание DataFrame
df = pd.DataFrame({
    'audio_path': [os.path.join(audio_dir, file) for file in audio_files]
})

# Разделение на обучающую и валидационную выборки
train_df, validation_df = train_test_split(df, test_size=0.2, random_state=42)

# Загрузка предварительно обученной модели и токенизатора
model_name = "ai4bharat/indicwav2vec_v1_bengali"
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Функция для чтения аудиофайла
def read_audio(file_path):
    speech, _ = sf.read(file_path)
    return speech

# Функция для подготовки данных
def prepare_data(df):
    input_values = []
    for _, row in df.iterrows():
        speech = read_audio(row['audio_path'])
        input_value = tokenizer(speech, return_tensors="pt", padding="longest").input_values[0]
        input_values.append(input_value)

    return torch.stack(input_values)

# Подготовка данных
train_input_values = prepare_data(train_df)
validation_input_values = prepare_data(validation_df)

# Настройки обучения
training_args = TrainingArguments(
  output_dir="/kaggle/working/results",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=1,
  save_steps=400,
  eval_steps=400,
  logging_steps=400
)

# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_input_values,
    eval_dataset=validation_input_values
)

# Запуск обучения
trainer.train()

# Сохранение модели
model.save_pretrained("/kaggle/working/fine_tuned_model")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_values,attention_mask,output_attentions,output_hidden_states,return_dict,labels,label,labels,label_ids.